In [ ]:
n_orders=1000 # consider only `n_orders` random orders (None - considers all)

# Imply the cube parameters from real data
The goal is to prepare the required parameters in order to generate synthetic cube data driven by the patterns from a real dataset.

Let $S$ be a sequence of categories (segments) (e.g. for movies ```['adventure', 'comedies',...]```.

The synthetic cube data model works at an agent level. 
The main assumption is that (1) every agent $a$ has a custom preferred order `segment_order[a]` of segments and (2) probability of approving a movie in agent's most preferred segment depends only on the location of the segment in `segment_order[a]` and is the same across all agents (we call it `cube_proba`), e.g., `cube_proba=[.5,.1,.1,.1,.1,.025,.025,.025,.025]` for nine segments.

This assumption allows for generation of the agent's approvals by, first, assigning probability density to a any resource x

That is, one might like comedies best, then adventure movies; while another one might like 'adventure' movies first then 'comedies'
share a _common probability distribution_ over _agent-specific segment order_

The input is the raw approval data (e.g., derived from movie ratings, where a movie is approved if it's rating $>=3$).


Let $P$ be a sequence of probabilities where $p_i$ is the probability that a given movie is in segment $S_i$, e.g., $p_0$ means probability that a movie is an 'adventure' movie.

Let $Q$ be a sequence $\in[0..1]$ where $q_i$ is the probability of segment $s_i$ occurring in the data.


The parameters to imply:
- cube_proba: 

Approach:
- retrieve the categorised movie data (movie - category)


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 

import pandas as pd
from es.data.movielens_data import MovielensData
from es.data_config import *
from es.elections import *


## Retrieve the 'approve_df'

In [ ]:
def load_category_df():
    import zipfile
    with zipfile.ZipFile(f"{data_folder_movielens_raw}/ml-25m.zip") as z:
        with z.open(f"ml-25m/movies.csv") as f:
            df = pd.read_csv(f, header=0, delimiter=',', encoding="ISO-8859-1",
                             names=['product_id', 'title', 'cat_id'])


    # df = pd.read_csv("/Users/Shared/tmp/esearch/movielens_raw/ml-25m/movies.csv", encoding="ISO-8859-1")
    df['cat_id'] =df.cat_id.apply(lambda x: x.split("|"))
    df = df.explode('cat_id')
    return df

category_df = load_category_df()
category_df[:3]

In [ ]:
%%time
idata = MovielensData(data_movielens_25m)
frames = InstacartDataFrames.from_data(idata)
if n_orders is not None:
    frames.sample(n_orders, random_state=13, inplace=True)

print('creating approvals df')
%time df_approvals = pd.merge(frames.df_order_products, category_df[['product_id', 'cat_id']], on='product_id', how='inner')
df_approvals = df_approvals[['order_id', 'product_id', 'cat_id']]
df_approvals.columns = ['agent_id', 'resource_id', 'cat_id']
# df_approvals = df_approvals.query('agent_id<10000')
df_approvals

## Obtain `cat_proba`

Probability of approving a movie per category by an agent, sorted descending.

Or: the distribution of frequency-ordered categories


Example: 
- `cat_proba[0]` means the average probability of the most frequently chosen resource category across all agents
- `cat_proba[1]` means the average ... of the second most frequent, etc...

In [ ]:
%%time
from collections import Counter

def list2probas_func(topn):
    def f(ids):
        cnt = Counter()
        for id in ids:
            cnt[id] += 1
        x = [count for _, count in cnt.most_common(topn)]
        return np.divide(x, np.sum(x))
    return f

def compute_proba(topn, group, ids):

    df = df_approvals.groupby(group)[ids].apply(list).reset_index(name=ids)
    df['probas'] = df[ids].apply(list2probas_func(topn))

    display(df[:3])

    df2 = pd.DataFrame(df.probas.tolist(), index=df.index)

    display(df2[:3])
    df2.hist(layout=(topn, 1), figsize=(100,20), sharex=True)

    #     max probability gets chosen among all
    proba = df2.median()
    # normalise
    proba = np.divide(proba, np.sum(proba))
    return [round(x, 3) for x in proba.tolist()]
    
cat_proba = compute_proba(topn=9, group='agent_id', ids = 'cat_id')
cat_proba

## obtain `res_proba`

Probability of approving a movie per category, sorted descending.

Or: distribution of frequency-ordered resources within categories

 (the pattern of resources within categories) [.5,.2,.1,...]

In [ ]:
%%time
res_proba = compute_proba(topn=25, group='cat_id', ids = 'resource_id')
res_proba

In [ ]:
import seaborn as sns

print('cat_proba =', cat_proba)
df = pd.DataFrame(zip(range(len(cat_proba)), cat_proba), columns=['agent category index', 'category probability'])
sns.relplot(data=df, y=df.columns[1], x=df.columns[0], palette='pastel')
print('---')
print('res_proba =', res_proba)
df = pd.DataFrame(zip(range(len(res_proba)), res_proba), columns=['movie category index', 'movie probability'])
sns.relplot(data=df, y=df.columns[1], x=df.columns[0], palette='pastel')
